In [29]:
import pandas as pd
import requests
import json
import requests
import time
from datetime import datetime
from IPython.display import display, clear_output
import numpy as np

In [30]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [31]:
(0.001 / 0.0001)*(1000/8000)

1.25

In [32]:
target_diff = 0.001
stop_loss = 0.001
units = 2000
profit = (target_diff / 0.0001)*(units/8000)
loss = (stop_loss / 0.0001)*(units/8000)

In [33]:
filename_date = '100519'

layout = pd.read_csv('../data/' + filename_date + 'mds_out_layout.csv')
layout = layout['features'].values
data = pd.read_csv('../data/' + filename_date + 'mds_out.csv', usecols=layout)
data.shape

(16423, 595)

In [34]:
data['time'].min()

'2017-01-19 21:00:00'

In [35]:
data['time'].max()

'2019-09-24 19:00:00'

In [36]:
num_cols = ['ask_c', 'bid_c']

In [37]:
data['time'] = pd.to_datetime(data['time'])
data = data.sort_values(by='time')
data = data.drop_duplicates(subset=['time'])
data = data.reset_index(drop=True)
data['index'] = data.index
for n in num_cols:
    data[n] = pd.to_numeric(data[n])
data['target_sl'] = data['ask_c'] + stop_loss
data['target_tp'] = data['bid_c'] - target_diff
data = data[~data['target_tp'].isnull()]

In [38]:
data['diff'] = data['target_tp'] - data['target_sl']
data['diff'].describe()

count    16423.000000
mean        -0.002211
std          0.000139
min         -0.003700
25%         -0.002200
50%         -0.002180
75%         -0.002160
max         -0.002100
Name: diff, dtype: float64

In [39]:
# data[['index', 'time', 'volume', 'bid_o', 'bid_h','bid_l','bid_c', 'ask_o', 'ask_h','ask_l','ask_c', 'ask_prev_c', 'bid_prev_c', 'target_tp', 'target_sl', 'g', 'l', 'l_time']].head(40)

In [40]:
data['g']=np.NaN
data['l']=np.NaN
deep=len(data.index)
irange = data['index'].values
tp_good = False
sl_good = False

for i in irange:

#     while tp_good == False and sl_good == False:
    show = (i / len(irange))*100
    show = str(round(show, 2))
    show = show + ' - ' + str(datetime.now())
    display(show)
    tp_good = False
    sl_good = False
#     print(i)

    for r in irange:
#             print(i, r, tp_good, sl_good)
        if r > i:
            if tp_good == False and sl_good == False:
                if  data[data['index'] == r]['ask_h'].values[0] >= data[data['index'] == i]['target_sl'].values[0] and not tp_good:
                    data.at[i, 'g'] = data[data['index'] == r]['index'].values[0]
                    data.at[i, 'g_time'] = data[data['index'] == r]['time'].values[0]
                    tp_good = True
                else:
                    data.at[i, 'g'] = None
                    data.at[i, 'g_time'] =None

                if  data[data['index'] == r]['bid_l'].values[0] <= data[data['index'] == i]['target_tp'].values[0] and not sl_good:
                    data.at[i, 'l'] = data[data['index'] == r]['index'].values[0]
                    data.at[i, 'l_time'] = data[data['index'] == r]['time'].values[0]
                    sl_good = True
                else:  
                    data.at[i, 'l'] = None     
                    data.at[i, 'l_time'] = None  
            else:
                break

    clear_output()    

#     print(datetime.now())

In [41]:
# # START WITH FIRST TARGET INDEX BEC TIME ISSUE

# if tp_good:
#     target_index = data['g'].max()
#     good_data = data[~data['g'].isnull()].copy()

# if sl_good:
#     target_index = data['l'].max()
#     good_data = data[~data['g'].isnull()].copy()

# tp_good = False
# sl_good = False
# done = False

# counter = 1
# while not done:
#     show = (target_index / len(irange))*100
#     show = str(round(show, 2))
#     show = show + ' - ' + str(datetime.now())
#     display(show)
#     tp_good = False
#     sl_good = False

#     for r in irange:
#         if r > target_index:
#             if tp_good == False and sl_good == False:
#                 if  data[data['index'] == r]['ask_h'].values[0] >= data[data['index'] == target_index]['target_tp'].values[0] and not tp_good:
#                     data.at[target_index, 'g'] = data[data['index'] == r]['index'].values[0]
#                     data.at[target_index, 'g_time'] = data[data['index'] == r]['time'].values[0]
#                     tp_good = True
#                 else:
#                     data.at[target_index, 'g'] = None
#                     data.at[target_index, 'g_time'] =None


#                 if  data[data['index'] == r]['bid_l'].values[0] <= data[data['index'] == target_index]['target_sl'].values[0] and not sl_good:
#                     data.at[target_index, 'l'] = data[data['index'] == r]['index'].values[0]
#                     data.at[target_index, 'l_time'] = data[data['index'] == r]['time'].values[0]
#                     sl_good = True

#                 else:  
#                     data.at[i, 'l'] = None     
#                     data.at[i, 'l_time'] = None  
#             else:
#                 break
    
#     if tp_good == True:
#         target_index = data['g'].max()
#         try:
#             temp = data[data['g'] == target_index].copy()
#             temp2 = data[data['index'] == target_index].copy()

#             if counter == 1:
#                 good_data = temp
#                 counter += 1
#             else:
#                 good_data = good_data.append(temp)

#             if target_index == data['index'].max():
#                 done = True
#         except IndexError:
#             done = True

#     elif sl_good == True:
#         target_index = data['l'].max()
#         try:
#             temp = data[data['l'] == target_index].copy()
#             temp2 = data[data['index'] == target_index].copy()

#             if counter == 1:
#                 good_data = temp
#                 counter += 1
#             else:
#                 good_data = good_data.append(temp)

#             if target_index == data['index'].max():
#                 done = True
#         except IndexError:
#             done = True
            
#     else:
#         done = True


#     clear_output()    

In [42]:
good_mask = (data['g'].isnull()) & (data['l'].isnull())
good_data = data[~good_mask].copy()
good_data.shape

(16419, 603)

In [43]:
good_data[['time', 'volume', 'bid_o', 'bid_h','bid_l','bid_c', 'ask_o', 'ask_h','ask_l','ask_c', 'target_tp', 'target_sl', 'g', 'l', 'l_time']].head(10)

,time,volume,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,target_tp,target_sl,g,l,l_time
0,2017-01-19 21:00:00,3959,1.33196,1.33261,1.33176,1.33252,1.33229,1.33284,1.33201,1.33278,1.33152,1.33378,NaN,1.0,2017-01-19 22:00:00
1,2017-01-19 22:00:00,2616,1.33257,1.33257,1.32843,1.32871,1.33282,1.33283,1.32867,1.32899,1.32771,1.32999,2.0,NaN,NaT
2,2017-01-19 23:00:00,1768,1.32864,1.33050,1.32833,1.32970,1.32895,1.33073,1.32859,1.32993,1.32870,1.33093,NaN,3.0,2017-01-20 00:00:00
3,2017-01-20 00:00:00,975,1.32972,1.32976,1.32851,1.32941,1.32991,1.32996,1.32877,1.32965,1.32841,1.33065,5.0,NaN,NaT
4,2017-01-20 01:00:00,1385,1.32945,1.33026,1.32945,1.32989,1.32967,1.33051,1.32966,1.33012,1.32889,1.33112,8.0,NaN,NaT
5,2017-01-20 02:00:00,2135,1.32991,1.33077,1.32966,1.33060,1.33010,1.33102,1.32988,1.33084,1.32960,1.33184,NaN,6.0,2017-01-20 03:00:00
6,2017-01-20 03:00:00,2472,1.33059,1.33060,1.32933,1.32987,1.33085,1.33086,1.32959,1.33012,1.32887,1.33112,8.0,NaN,NaT
7,2017-01-20 04:00:00,2779,1.32990,1.33036,1.32898,1.33005,1.33014,1.33059,1.32918,1.33028,1.32905,1.33128,8.0,NaN,NaT
8,2017-01-20 05:00:00,2430,1.33006,1.33244,1.32925,1.33242,1.33028,1.33266,1.32949,1.33264,1.33142,1.33364,9.0,NaN,NaT
9,2017-01-20 06:00:00,2937,1.33238,1.33436,1.33229,1.33367,1.33259,1.33459,1.33251,1.33388,1.33267,1.33488,10.0,NaN,NaT


In [44]:
def get_target(value):
    if value > 0:
        return 1
    else:
        return 0

In [45]:
# max_date = data['time'].max()
good_data['g_time'] = pd.to_datetime(good_data['g_time'])
good_data['l_time'] = pd.to_datetime(good_data['l_time'])

good_data['g'] = pd.to_numeric(good_data['g'])
good_data['l'] = pd.to_numeric(good_data['l'])

good_data['g'] = good_data['g'].fillna(0)
good_data['l'] = good_data['l'].fillna(0)

# data['g'] = data['g'].fillna(data['index'].max())
# data['l'] = data['l'].fillna(data['index'].max())

good_data['long_target'] = good_data['g'].apply(get_target)
good_data['short_target'] = good_data['l'].apply(get_target)

print(good_data['long_target'].value_counts())
print()
print(good_data['short_target'].value_counts())

1    8687
0    7732
Name: long_target, dtype: int64

1    8493
0    7926
Name: short_target, dtype: int64


In [46]:
10101*3

30303

In [47]:
6300*4.5

28350.0

In [48]:
# good_data['g_time_shift'] = good_data['g_time'].shift(1)
# good_data['g_time_shift'] = pd.to_datetime(good_data['g_time_shift'] )
good_data['time'] = pd.to_datetime(good_data['time'])
# good_data['g_diff'] = good_data['g_time'] - good_data['g_time_shift']
# good_data['g_diff'] = good_data['g_diff'].dt.seconds

In [49]:
good_data['order'] = good_data.index
t1 = good_data[good_data['g'] != 0.0].drop_duplicates(subset='g')
t2 = good_data[good_data['l'] != 0.0].drop_duplicates(subset='l')
good_data = pd.concat([t1, t2], axis=0)
good_data= good_data.sort_values(by='order')
good_data['same_time'] = good_data['g_time'] == good_data['l_time']
good_data['same_time'] = good_data['same_time'].apply(lambda x: 1 if x else 0)
good_data = good_data[good_data['same_time'] == 0]
good_data.shape

(9466, 607)

In [50]:
pos = good_data[good_data['short_target'] == 1].copy()
print(pos.shape)
neg = good_data[good_data['short_target'] == 0].copy()

pos['time_diff'] = (pos['l_time'] - pos['time']) / np.timedelta64(1, 'h')
pos_bad = pos[pos['time_diff'] > 4].copy()
pos_bad['short_target'] = 0

pos = pos[pos['time_diff'] <= 4]

pos.drop(['time_diff'], axis=1, inplace=True)
pos_bad.drop(['time_diff'], axis=1, inplace=True)

good_data = pd.concat([pos, neg, pos_bad], axis=0)
good_data.shape

(4726, 607)


(9466, 607)

In [51]:
good_data.to_csv('../data/' + filename_date + '_mds_out_final_v7_USDCAD_short.csv', index=False, header=True)

In [52]:
good_data[['time', 'target_tp', 'target_sl', 'g', 'l', 'g_time', 'l_time', 'long_target', 'short_target']].head()

,time,target_tp,target_sl,g,l,g_time,l_time,long_target,short_target
0,2017-01-19 21:00:00,1.33152,1.33378,0.0,1.0,NaT,2017-01-19 22:00:00,0,1
2,2017-01-19 23:00:00,1.32870,1.33093,0.0,3.0,NaT,2017-01-20 00:00:00,0,1
5,2017-01-20 02:00:00,1.32960,1.33184,0.0,6.0,NaT,2017-01-20 03:00:00,0,1
14,2017-01-20 11:00:00,1.33656,1.33873,0.0,15.0,NaT,2017-01-20 12:00:00,0,1
17,2017-01-20 14:00:00,1.33425,1.33645,0.0,18.0,NaT,2017-01-20 15:00:00,0,1


In [53]:
good_data[['time', 'target_tp', 'target_sl', 'g', 'l', 'g_time', 'l_time', 'long_target', 'short_target']].tail(10)

,time,target_tp,target_sl,g,l,g_time,l_time,long_target,short_target
16138,2019-09-09 01:00:00,1.31572,1.31789,0.0,16144.0,NaT,2019-09-09 07:00:00,0,0
16142,2019-09-09 05:00:00,1.31483,1.31700,0.0,16148.0,NaT,2019-09-09 11:00:00,0,0
16176,2019-09-10 15:00:00,1.31395,1.31611,0.0,16186.0,NaT,2019-09-11 01:00:00,0,0
16198,2019-09-11 13:00:00,1.31880,1.32101,0.0,16205.0,NaT,2019-09-11 20:00:00,0,0
16248,2019-09-13 15:00:00,1.32714,1.32932,0.0,16252.0,NaT,2019-09-15 18:00:00,0,0
16271,2019-09-16 13:00:00,1.32344,1.32561,0.0,16277.0,NaT,2019-09-16 19:00:00,0,0
16309,2019-09-18 03:00:00,1.32546,1.32762,0.0,16314.0,NaT,2019-09-18 08:00:00,0,0
16326,2019-09-18 20:00:00,1.32884,1.33104,0.0,16331.0,NaT,2019-09-19 01:00:00,0,0
16394,2019-09-23 15:00:00,1.32487,1.32703,0.0,16412.0,NaT,2019-09-24 09:00:00,0,0
16396,2019-09-23 17:00:00,1.32497,1.32742,0.0,16408.0,NaT,2019-09-24 05:00:00,0,0


In [54]:
shorts = good_data[good_data['short_target'] == 1].copy()
shorts['time'] = pd.to_datetime(shorts['time'])
shorts['l_time'] = pd.to_datetime(shorts['l_time'] )
shorts['time_diff'] = shorts['l_time'] - shorts['time']
shorts['time_diff'].head() 

0    01:00:00
2    01:00:00
5    01:00:00
14   01:00:00
17   01:00:00
Name: time_diff, dtype: timedelta64[ns]

In [55]:
shorts['time_diff'] = shorts['time_diff']  / np.timedelta64(1, 'h')

In [56]:
shorts['time_diff'].describe()

count    4088.000000
mean        1.687133
std         0.938642
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         4.000000
Name: time_diff, dtype: float64